환경부 2020년 전기자동차 보급 및 충전인프라 구축사업 충전인프라 설치운영지침에 따르면   
해당 장소에 기 설치된 충전기가 없는 경우
#### 해당 장소의 주차단위구획 수× (0.01 + 해당 시‧도의 최근 3년간 신규 차량등록대수 분의 신규 전기차 등록대수) 까지 설치가 가능하다

전라남도청이 공개한 2019년 말 기준 광양시의 지난 3년간의 자동차 등록 현황 대수는    
87,990대(2019년 말) - 78,235대(2016년 9월 기준) = 9755대이다.

In [39]:
%cd E:\yujin\INHA\data

import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from pandas import Series, DataFrame
from math import floor
import warnings
warnings.filterwarnings(action='ignore')

E:\yujin\INHA\data


## 1) 주차장데이터에 현재 설치되어 있는 충전기가 포함되어 있는지 확인

In [40]:
chargingSpot = pd.read_csv('01.광양시_충전기설치현황.csv')
parkingLot = pd.read_csv('02.광양시_주차장_공간정보.csv')
gyboundary = gpd.read_file('20.광양시_행정경계(읍면동).geojson')
administrative = gyboundary.ADM_DR_NM.tolist()

In [41]:
#현재 설치되어 있는 충전기는 주차장 데이터에 포함되어 있지 않음
충전기_주차장 = []
for i in range(len(chargingSpot)):
    for j in range(len(parkingLot)):
        if (chargingSpot['lon'][i] == parkingLot['lon'][j]) and (chargingSpot['lat'][i] == parkingLot['lat'][j]):
            충전기_주차장.append(j)
충전기_주차장

[]

## 2) 최근 3년간 신규 전기차 등록대수

In [42]:
#주차장 전처리
parkingLot = pd.read_csv('02.광양시_주차장_공간정보.csv')
parkingLot['lon'] = parkingLot['lon'].astype(float)
parkingLot['lat'] = parkingLot['lat'].astype(float)
parkingLot['geometry'] = parkingLot.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
parkingLot = gpd.GeoDataFrame(parkingLot, geometry='geometry')
parkingLot = parkingLot[['면적','주차면수','geometry']]
parkingLot.rename(columns={"면적":"area"}, inplace=True)

pllist = []
for i in range(len(parkingLot)):
    for j in range(len(gyboundary)):
        if parkingLot.geometry[i].within(gyboundary.geometry[j]) == True:
            pllist.append(gyboundary.ADM_DR_NM[j])

parkingLot['administration'] = pllist
parkingLot

,area,주차면수,geometry,administration
0,4189.0,161.0,POINT (127.58688 34.97032),광양읍
1,2968.0,125.0,POINT (127.69543 34.93625),중마동
2,2836.0,89.0,POINT (127.58459 34.97564),광양읍
3,1980.0,52.0,POINT (127.58068 34.97013),광양읍
4,1970.0,64.0,POINT (127.58071 34.97252),광양읍
...,...,...,...,...
230,264.0,10.0,POINT (127.60482 35.07921),옥룡면
231,1212.0,60.0,POINT (127.61342 35.07067),옥룡면
232,2381.0,96.0,POINT (127.61177 35.00276),옥룡면
233,2090.0,66.0,POINT (127.67472 35.09239),진상면


In [43]:
#충전기 전처리
chargingSpot = pd.read_csv('01.광양시_충전기설치현황.csv')
chargingSpot['lon'] = chargingSpot['lon'].astype(float)
chargingSpot['lat'] = chargingSpot['lat'].astype(float)
chargingSpot['geometry'] = chargingSpot.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
chargingSpot = gpd.GeoDataFrame(chargingSpot, geometry='geometry')
chargingSpot['급속/완속'] = np.where(chargingSpot['급속/완속']=='민간',0,1) #급속 = 0, 완속 = 1
chargingSpot = chargingSpot[['급속/완속','이용대수','geometry']]

cllist = []
for i in range(len(chargingSpot)):
    for j in range(len(gyboundary)):
        if chargingSpot.geometry[i].within(gyboundary.geometry[j]) == True:
            cllist.append(gyboundary.ADM_DR_NM[j])

chargingSpot['administration'] = cllist
chargingSpot

,급속/완속,이용대수,geometry,administration
0,1,2,POINT (127.56830 34.96300),광양읍
1,1,1,POINT (127.58332 34.98033),광양읍
2,1,5,POINT (127.68901 34.95376),중마동
3,1,2,POINT (127.71671 34.96403),광영동
4,1,2,POINT (127.73244 34.94044),금호동
5,1,1,POINT (127.58244 34.96333),광양읍
6,1,2,POINT (127.75098 34.96251),진월면
7,1,1,POINT (127.76007 34.94120),태인동
8,1,2,POINT (127.72513 34.96215),광영동
9,1,1,POINT (127.58061 34.97306),광양읍


In [44]:
ElecCar = pd.read_csv('06.광양시_전기차보급현황(연도별,읍면동별).csv')
adm = ElecCar.행정구역.unique()

x = []
y = []
z = []
q = []
for i in range(len(ElecCar)):
    if ElecCar.기준년도[i] == 2017:
        x.append(ElecCar.보급현황[i])
    elif ElecCar.기준년도[i] == 2018:
        y.append(ElecCar.보급현황[i])
    elif ElecCar.기준년도[i] == 2019:
        z.append(ElecCar.보급현황[i])
    else:
        q.append(ElecCar.보급현황[i])


data = {'administration': adm, '2017' : x, '2018' : y , "2019" : z, '2020': q}
ElecCar = pd.DataFrame(data)
ElecCar['2017'].astype('int')
ElecCar['2018'].astype('int')
ElecCar['2019'].astype('int')
ElecCar['2020'].astype('int')
ElecCar

,administration,2017,2018,2019,2020
0,광양읍,17,1,44,40
1,봉강면,0,0,4,0
2,옥룡면,3,0,3,5
3,옥곡면,3,0,2,3
4,진상면,1,0,2,2
5,진월면,1,0,2,2
6,다압면,0,0,0,0
7,골약동,2,1,1,0
8,중마동,17,8,27,26
9,광영동,3,0,6,3


In [45]:
elist = []
for i in range(len(ElecCar)):
    a = 0
    for j in range(1, len(ElecCar.columns)-1):
        if ElecCar.loc[i][j] < ElecCar.loc[i][j+1]:
            x = ElecCar.loc[i][j+1] - ElecCar.loc[i][j]
            a += x
    elist.append(a)

print(elist) 

[43, 4, 5, 3, 2, 2, 0, 0, 19, 6, 0, 9]


In [48]:
#각 읍면동별 3년간 신규 전기차 등록대수
administrative.remove('광양읍')
administrative.insert(0, '광양읍')
data = {'administration': administrative, 'supply' : elist}
ElecCar_Supply = pd.DataFrame(data)
total = sum(ElecCar_Supply.supply)
ElecCar_Supply

,administration,supply
0,광양읍,43
1,봉강면,4
2,옥룡면,5
3,옥곡면,3
4,진상면,2
5,진월면,2
6,다압면,0
7,골약동,0
8,중마동,19
9,광영동,6


In [49]:
#광양시 3년간 신규 전기차 등록대수
total_new_eleccar = ElecCar_Supply['supply'].sum()
total_new_eleccar

93

## 3) 식에 값 대입해서 weight계산

In [50]:
#해당 장소의 주차단위구획 수 × (0.01 + 해당 시‧도의 최근 3년간 신규 차량등록대수 분의 신규 전기차 등록대수)

weight_lst = []
for i in parkingLot['주차면수']:
    x = i * (0.01 + (total_new_eleccar/9755))
    weight_lst.append(x)
    

parkingLot['weight'] = weight_lst
parkingLot

,area,주차면수,geometry,administration,weight
0,4189.0,161.0,POINT (127.58688 34.97032),광양읍,3.144905
1,2968.0,125.0,POINT (127.69543 34.93625),중마동,2.441697
2,2836.0,89.0,POINT (127.58459 34.97564),광양읍,1.738488
3,1980.0,52.0,POINT (127.58068 34.97013),광양읍,1.015746
4,1970.0,64.0,POINT (127.58071 34.97252),광양읍,1.250149
...,...,...,...,...,...
230,264.0,10.0,POINT (127.60482 35.07921),옥룡면,0.195336
231,1212.0,60.0,POINT (127.61342 35.07067),옥룡면,1.172014
232,2381.0,96.0,POINT (127.61177 35.00276),옥룡면,1.875223
233,2090.0,66.0,POINT (127.67472 35.09239),진상면,1.289216


In [51]:
#37,38 주차장은 주차면수가 nan이어서 weight도 nan이 나옴.
parkingLot[parkingLot['weight'].isnull()]

,area,주차면수,geometry,administration,weight
37,3143.0,NaN,POINT (127.60638 34.94442),광양읍,NaN
38,2692.0,NaN,POINT (127.60644 34.94261),광양읍,NaN
